 # Multimodal RAG Model using Byaldi

Установим требуемые нам библиотеки: 
* `byaldi` - высокоуровневый [фреймворк](https://github.com/AnswerDotAI/byaldi) непосредственно для работы с моделями ColPali
* `pdf2image` - для перевода `.pdf`-файлов в изображения
* `poppler-utils` - для работы `pdf2image`
* `Spire.Doc` - для перевода `.docx`-файлов в формат `.pdf`

In [5]:
!pip install byaldi
!pip install poppler-utils
!pip install -q pdf2image flash-attn -q
!pip install Spire.Doc -q

  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [22 lines of output]
  fatal: not a git repository (or any of the parent directories): .git
  C:\Temp\pip-install-5ntab2oj\flash-attn_b7fd137a96814c60a714a71dbb79694e\setup.py:99: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
    warnings.warn(
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Temp\pip-install-5ntab2oj\flash-attn_b7fd137a96814c60a714a71dbb79694e\setup.py", line 183, in <module>
      CUDAExtension(
    File "C:\Users\Сергей\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\cpp_extension.py", line 1078, in CUDAExtension
      libra

Определим папку, где у нас лежат первоначальные данные `input_folder`, а так же папку, куда мы переведём файлы непосредственно для работы с RAG-системой `working_folder`.

In [6]:
from byaldi import RAGMultiModalModel
import torch
from pdf2image import convert_from_path
import os
import shutil
from spire.doc import *
from spire.doc.common import *


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
input_folder = '/kaggle/input/nornikel-2024/'
working_folder = '/kaggle/working/nornikel-2024/'

if not os.path.exists(working_folder):
    os.makedirs(working_folder)

In [7]:
def convert_doc_to_pdf(doc_path, pdf_path):
    document = Document()
    document.LoadFromFile(doc_path)
    document.SaveToFile(pdf_path, FileFormat.PDF)
    document.Close()

In [8]:
all_documents = os.listdir(input_folder)

for file in all_documents:
    file_path = os.path.join(input_folder, file)
    if file_path.endswith('.docx') or file_path.endswith('.doc'):
        file_path = convert_doc_to_pdf(file_path, working_folder + file[:-4] + 'pdf')
        print(f"Копирован и переведён в формат .pdf файл: {file[:-4] + 'pdf'}")
    else:
        shutil.copy(file_path, working_folder)
        print(f"Копирован файл: {file}")

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: '/kaggle/input/nornikel-2024/'

Определим метод RAGMultiModalModel из библиотеки byaldi, который отвечает за создание мультимодальной RAG-системы с помощью модели ColPali.

In [ ]:
RAG = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2", device=device)

Создадим нашу векторную базу из всех документов, которые у нас есть.

In [ ]:
RAG.index(
    input_path=working_folder,
    index_name='nornikel_index',
    store_collection_with_index=False,
    overwrite=True
)

Сформируем текстовый запрос, чтобы убедиться, что модель работает. Определим k страниц, где есть релевантная информация.

In [ ]:
text_query = "Расскажи про рынок блирриантов"
results = RAG.search(text_query, k=5)

In [ ]:
for result in results:
    doc_id = result['doc_id']
    page_num = result['page_num']
    score = result['score']

    print(f"Документ ID: {doc_id}, Страница: {page_num}, Релевантность: {score}")

Сохраним нашу векторную базу для дальнейшей эксплуатации.

In [ ]:
from zipfile import ZipFile

byaldi_folder = '/kaggle/working/.byaldi'
archive_name = '/kaggle/working/byaldi.zip'

if os.path.exists(byaldi_folder):
    shutil.make_archive(archive_name.replace('.zip', ''), 'zip', byaldi_folder)
    print(f"Архив создан: {archive_name}")
else:
    print(f"Папка {byaldi_folder} не найдена.")
